In [ ]:
import numpy as np
import tensorflow as tf
from pandas import read_csv
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from scikeras.wrappers import KerasClassifier # pip install scikeras
from keras.constraints import MaxNorm

In [6]:
datos = read_csv('breast-cancer.csv', header=None)
# array de numpy
matriz = datos.values
# Entradas (X) y salida (y)
X = matriz[:, :-1]
y = matriz[:,-1]
# Tipo str para todas las variables
X = X.astype(str)
# Vectorización de la tabla y
y = y.reshape((len(y), 1))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
oe = OrdinalEncoder()
oe.fit(X_train)
X_train_enc = oe.transform(X_train)
X_test_enc = oe.transform(X_test)
le = LabelEncoder()
le.fit(y_train)
y_train_enc = le.transform(y_train)
y_test_enc = le.transform(y_test)

/home/julio/.local/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/julio/.local/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/home/julio/.local/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [10]:
def create_model(neurons):
    # create model
    model = Sequential()
    model.add(Dense(neurons, input_dim=X_train_enc.shape[1], activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model


model = KerasClassifier(model=create_model, epochs=100, batch_size=10, verbose=0, neurons=1)

# Definir los hiperparámetros a ajustar
param_dist = {
    'neurons': [1, 5, 10, 15, 20]
}


grid = GridSearchCV(estimator=model, param_grid=param_dist, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train_enc, y_train_enc)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

/home/julio/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/julio/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/julio/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regula

Best: 0.719298 using {'neurons': 5}
0.714912 (0.006203) with: {'neurons': 1}
0.719298 (0.032822) with: {'neurons': 5}
0.679825 (0.032822) with: {'neurons': 10}
0.684211 (0.018608) with: {'neurons': 15}
0.640351 (0.040674) with: {'neurons': 20}


In [9]:
best_params = grid_result.best_params_
neurons_best = best_params['neurons']

# Crea el modelo con los mejores parámetros
best_model = create_model(neurons=neurons_best)

# Entrenar el modelo en el conjunto de entrenamiento
history = best_model.fit(X_train_enc, y_train_enc, epochs=30, validation_split=0.2, batch_size=10) 

Epoch 1/30


/home/julio/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6753 - loss: 0.6610 - val_accuracy: 0.6957 - val_loss: 0.6115
Epoch 2/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7461 - loss: 0.5918 - val_accuracy: 0.6957 - val_loss: 0.5947
Epoch 3/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7353 - loss: 0.5671 - val_accuracy: 0.6957 - val_loss: 0.5799
Epoch 4/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7081 - loss: 0.5716 - val_accuracy: 0.6957 - val_loss: 0.5716
Epoch 5/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7605 - loss: 0.5249 - val_accuracy: 0.6957 - val_loss: 0.5643
Epoch 6/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7607 - loss: 0.5167 - val_accuracy: 0.6957 - val_loss: 0.5633
Epoch 7/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7202 - loss: 0.5365 - val_accuracy: 0.6957 - val_loss: 0.5705
Epoch 8/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6995 - loss: 0.5315 - val_accuracy: 0.6957 - val_loss: 0.5717
Epo